In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gemini-2.5-flash", model_provider="google_genai")

E0000 00:00:1758484505.178179   17627 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


**read about reducer functions**

In [ ]:
from typing import TypedDict
from typing_extensions import Annotated
from langgraph.graph import StateGraph, START, END
#reducer functio -> adds some kind of validation to messages
from langgraph.graph.message import add_messages

class State(TypedDict):
    messages: Annotated[list, add_messages]

def chatbot(state: State) -> State:
    return {'messages': [llm.invoke(state['messages'])]}

builder = StateGraph(State)
builder.add_node('chatbot_node', chatbot)

builder.add_edge(START, "chatbot_node")
builder.add_edge("chatbot_node", END)

graph = builder.compile()

In [15]:
message = {'role': 'user', 'content':'Who is the current president of the United States?'}
response = graph.invoke({'messages':[message]})

response["messages"]

[HumanMessage(content='Who is the current president of the United States?', additional_kwargs={}, response_metadata={}, id='5f298d7c-3f25-4608-b174-b8b8e842e8f1'),
 AIMessage(content='The current president of the United States is **Joe Biden**.\n\nHe took office on January 20, 2021.', additional_kwargs={}, response_metadata={'prompt_feedback': {'block_reason': 0, 'safety_ratings': []}, 'finish_reason': 'STOP', 'model_name': 'gemini-2.5-flash', 'safety_ratings': []}, id='run--8127e370-1c39-45ad-a668-3162b336dec5-0', usage_metadata={'input_tokens': 11, 'output_tokens': 139, 'total_tokens': 150, 'input_token_details': {'cache_read': 0}, 'output_token_details': {'reasoning': 111}})]

In [17]:
state = None

while True:
    in_message = input("You : ")

    if in_message.lower() in {"quit", "exit"}:
        break

    if state == None:
        state: State = {
            "messages": [{"role": "user", "content": in_message}]
        }
    else:
        state["messages"].append({"role": "user", "content": in_message})


    state = graph.invoke(state)
    print("Bot: ", state["messages"][-1].content)

Bot:  It is **2024**.
Bot:  The current F1 World Drivers' Champion (WDC) is **Max Verstappen**.

He won his third consecutive title in the **2023** season.
Bot:  Max Verstappen is not married, so he does not have a wife.

However, his long-term partner is **Kelly Piquet**.

She is a Brazilian model, columnist, and blogger, and the daughter of three-time Formula 1 World Champion Nelson Piquet. She also has a daughter, Penelope, from a previous relationship with former F1 driver Daniil Kvyat, and Max is her stepfather.
